In [166]:
import folium
import pandas as pd
import geopandas as gpd

In [167]:
import pandas
url = "http://localhost:5001/generate"
data = pandas.read_json(url)

In [168]:
m = folium.Map(location=[48.853,2.35], tiles="OpenStreetMap", zoom_start=12)

In [169]:
for index, row in data.iterrows():
    folium.Marker([row['lat'],row['lon']], popup='<i>'+str(row['id_roi'])+'</i>', tooltip=str(row['id_roi']), icon=folium.Icon()).add_to(m)

In [170]:
m